# 10 – Time Series Quickstart: ml_tabular Template

This notebook shows how to use the **ml_tabular** template for a simple time-series forecasting task.

We will:

1. Load configuration from a YAML file (time-series baseline)
2. Load and inspect the raw time-series data
3. Build a `TimeSeriesSequenceDataset` and PyTorch `DataLoader`s
4. Define a small GRU-based forecasting model
5. Train and evaluate using shared training utilities (`fit`, `evaluate`, `EarlyStopping`)
6. (Optionally) log runs and artifacts to MLflow

The aim is to demonstrate the **end-to-end path** for time-series using the same engineering principles as the tabular pipeline: config-driven, testable, and reproducible.

## 0. Prerequisites

We assume you have:

- Installed the project with extras:

  ```bash
  pip install -e .[dev,mlops]
  ```

- A baseline config at:

  - `configs/time_series/train_ts_baseline.yaml`

That config should define at least:

- `paths` (data, models)
- `training` (batch size, epochs, lr, etc.)
- `time_series` (dataset CSV, id column, time column, target, features, lookback, horizon, etc.)

This notebook is designed to be run from the **project root** (so relative paths work).

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from ml_tabular import (
    get_config,
    get_paths,
    get_logger,
    TimeSeriesSequenceDataset,
    train_one_epoch,
    evaluate,
    fit,
    EarlyStopping,
)

LOGGER = get_logger(__name__)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOGGER.info("Using device: %s", DEVICE)

PROJECT_ROOT = Path.cwd()
CONFIG_PATH = PROJECT_ROOT / "configs" / "time_series" / "train_ts_baseline.yaml"
assert CONFIG_PATH.exists(), f"Config not found: {CONFIG_PATH}"

## 1. Load configuration and inspect time-series settings

We use the same `AppConfig` / `get_config` pattern as in the tabular quickstart, but now read the **time-series** section of the config.

In [ ]:
cfg = get_config(config_path=CONFIG_PATH, env="dev", force_reload=True)
paths = get_paths(config_path=CONFIG_PATH, env="dev", force_reload=True)

cfg_dict = cfg.to_dict()
ts_cfg = cfg_dict.get("time_series", {})
ts_cfg

We expect `time_series` config to look roughly like:

```yaml
time_series:
  dataset_csv: "timeseries.csv"
  id_column: "series_id"           # or null for single series
  time_column: "timestamp"
  target_column: "y"
  feature_columns: ["x1", "x2", ...]
  lookback: 24                      # input sequence length
  horizon: 1                        # forecast horizon (e.g. predict next step)
  val_fraction: 0.2                 # optional; or explicit split timestamp
```

You can adapt field names, but the notebook assumes these semantics.

## 2. Load and inspect the dataset

We read the configured CSV and perform some basic sanity checks:

- Columns exist
- Time column can be parsed as datetime
- Data is sorted by (id, time) for sequence generation.

In [ ]:
dataset_csv = ts_cfg["dataset_csv"]
data_path = paths.data_dir / dataset_csv
assert data_path.exists(), f"Dataset not found: {data_path}"

df = pd.read_csv(data_path)
LOGGER.info("Loaded dataset with shape: %s", df.shape)
df.head()

In [ ]:
id_col = ts_cfg.get("id_column")  # may be None for single series
time_col = ts_cfg["time_column"]
target_col = ts_cfg["target_column"]
feature_cols = ts_cfg.get("feature_columns") or []

print("ID column:", id_col)
print("Time column:", time_col)
print("Target column:", target_col)
print("Feature columns:", feature_cols)

missing = [c for c in [time_col, target_col] + feature_cols if c not in df.columns]
print("Missing columns:", missing)
assert not missing, f"Config references columns not found in dataset: {missing}"

Parse the time column and sort by id/time (or just time for single-series data).

In [ ]:
df[time_col] = pd.to_datetime(df[time_col], errors="raise")

if id_col is not None:
    df = df.sort_values([id_col, time_col]).reset_index(drop=True)
else:
    df = df.sort_values(time_col).reset_index(drop=True)

df[[c for c in [id_col, time_col, target_col] if c is not None]].head(10)

We can also quickly inspect how many unique series we have (if `id_column` is set) and the time coverage.

In [ ]:
if id_col is not None:
    print("Number of series:", df[id_col].nunique())
    display(df.groupby(id_col)[time_col].agg(["min", "max", "count"]).head())
else:
    print("Single series dataset.")
    print("Time range:", df[time_col].min(), "->", df[time_col].max())
    print("Number of points:", len(df))

## 3. Train/validation split by time

For time series, we **do not shuffle** in time; instead we split chronologically.

We use a simple approach:

- Choose a `val_fraction` (e.g. 0.2)
- For each series (or globally for single-series data), use the first `(1 - val_fraction)` fraction for training and the rest for validation.

For this quickstart, we’ll apply a **global time split** (suitable for many scenarios). You can extend your pipeline for more sophisticated splitting if needed.

In [ ]:
val_fraction = float(ts_cfg.get("val_fraction", 0.2))
assert 0.0 < val_fraction < 1.0, "val_fraction should be in (0, 1)."

n_total = len(df)
n_train = int((1.0 - val_fraction) * n_total)

train_df = df.iloc[:n_train].copy()
val_df = df.iloc[n_train:].copy()

LOGGER.info("Train size: %d, Val size: %d", len(train_df), len(val_df))
train_df[[c for c in [id_col, time_col, target_col] if c is not None]].head()

## 4. Build `TimeSeriesSequenceDataset` and DataLoaders

We now turn the time-indexed dataframe into sequences for supervised learning. The semantics we assume for `TimeSeriesSequenceDataset`:

- Each sample is `(X_seq, y_target)` where:
  - `X_seq` has shape `(lookback, n_features)`
  - `y_target` has shape `(horizon,)` or scalar for horizon=1
- Sequences slide along time for each series independently (if `id_column` is set).

We’ll use the `lookback` and `horizon` from config.

In [ ]:
lookback = int(ts_cfg.get("lookback", 24))
horizon = int(ts_cfg.get("horizon", 1))

print("Lookback:", lookback)
print("Horizon:", horizon)

train_ds = TimeSeriesSequenceDataset.from_dataframe(
    train_df,
    id_column=id_col,
    time_column=time_col,
    feature_columns=feature_cols,
    target_column=target_col,
    lookback=lookback,
    horizon=horizon,
)

val_ds = TimeSeriesSequenceDataset.from_dataframe(
    val_df,
    id_column=id_col,
    time_column=time_col,
    feature_columns=feature_cols,
    target_column=target_col,
    lookback=lookback,
    horizon=horizon,
)

print("Train sequences:", len(train_ds))
print("Val sequences:", len(val_ds))
print("Metadata (train):", train_ds.metadata)

x_seq, y_target = train_ds[0]
print("X_seq shape:", x_seq.shape)
print("y_target shape:", y_target.shape)

In [ ]:
batch_size = cfg.training.batch_size if hasattr(cfg, "training") else 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
)

batch_x, batch_y = next(iter(train_loader))
print("Batch X shape (batch, seq_len, features):", batch_x.shape)
print("Batch y shape:", batch_y.shape)

## 5. Define a simple GRU-based forecasting model

For this quickstart, we build a small **GRU forecaster** inline in the notebook:

- Input: `(batch_size, seq_len, n_features)`
- GRU layers process the sequence
- We take the **last hidden state** and feed it through a linear layer to produce a forecast vector of size `horizon`.

This demonstrates how your template happily supports deep learning models with sequence structure, not just tabular MLPs.

In [ ]:
class GRUForecaster(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 64,
        num_layers: int = 1,
        horizon: int = 1,
        dropout: float = 0.0,
    ) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.horizon = horizon

        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.head = nn.Linear(hidden_dim, horizon)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass.

        Parameters
        ----------
        x:
            Tensor of shape (batch, seq_len, input_dim).
        """
        out, h_n = self.gru(x)
        # h_n: (num_layers, batch, hidden_dim) -> use last layer's hidden state
        last_hidden = h_n[-1]  # (batch, hidden_dim)
        preds = self.head(last_hidden)  # (batch, horizon)
        return preds


n_features = train_ds.metadata.get("n_features", len(feature_cols))
hidden_dim = int(ts_cfg.get("hidden_dim", 64))
num_layers = int(ts_cfg.get("num_layers", 1))
dropout = float(ts_cfg.get("dropout", 0.0))

model = GRUForecaster(
    input_dim=n_features,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    horizon=horizon,
    dropout=dropout,
).to(DEVICE)

model

## 6. Train and evaluate using shared utilities

We now tie everything together:

- Use `MSELoss` for regression-style forecasting
- Use Adam optimizer with hyperparameters from config
- Optionally use `EarlyStopping` based on validation loss
- Call `fit(...)` to run the training loop

This is exactly the same training infrastructure (`fit`, `EarlyStopping`) as the tabular pipeline, just with a sequence model and dataset.

In [ ]:
from torch.optim import Adam

learning_rate = cfg.training.learning_rate if hasattr(cfg, "training") else 1e-3
weight_decay = getattr(cfg.training, "weight_decay", 0.0) if hasattr(cfg, "training") else 0.0
num_epochs = cfg.training.num_epochs if hasattr(cfg, "training") else 10

loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Optional early stopping config
es_cfg = cfg_dict.get("training", {}).get("early_stopping", {})
if es_cfg:
    early_stopping = EarlyStopping(
        patience=int(es_cfg.get("patience", 5)),
        min_delta=float(es_cfg.get("min_delta", 1e-4)),
        mode=es_cfg.get("mode", "min"),
    )
else:
    early_stopping = None

history = fit(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    num_epochs=num_epochs,
    device=DEVICE,
    early_stopping=early_stopping,
)

history

Plot training vs validation loss to verify convergence and overfitting behaviour.

In [ ]:
import matplotlib.pyplot as plt

train_losses = history["train_losses"]
val_losses = history["val_losses"]

plt.figure(figsize=(6, 4))
plt.plot(train_losses, marker="o", label="train")
plt.plot(val_losses, marker="o", label="val")
plt.xlabel("Epoch")
plt.ylabel("MSE loss")
plt.title("Time-series training vs validation loss")
plt.legend()
plt.grid(True)
plt.show()

## 7. Inspect predictions vs ground truth

It’s useful to look at a few sequences and compare predicted vs actual targets. For horizon=1, this is just comparing next-step forecasts; for horizon>1, we can plot the forecast window vs true future values.

The quick visualization below assumes a univariate target with horizon=1 or a small horizon.

In [ ]:
model.eval()
with torch.no_grad():
    batch_x_val, batch_y_val = next(iter(val_loader))
    batch_x_val = batch_x_val.to(DEVICE)
    preds = model(batch_x_val).cpu().numpy()
    y_true = batch_y_val.numpy()

print("Preds shape:", preds.shape)
print("True shape:", y_true.shape)

# Take first few samples to visualize
n_plot = min(10, preds.shape[0])
t = np.arange(n_plot)

plt.figure(figsize=(6, 4))
if horizon == 1:
    plt.plot(t, y_true[:n_plot, 0], marker="o", label="true")
    plt.plot(t, preds[:n_plot, 0], marker="x", label="pred")
    plt.xlabel("Sample index (in validation batch)")
    plt.ylabel("Target")
    plt.title("Next-step forecast: true vs pred")
else:
    # For multi-step horizon, just compare first horizon element for each sample
    plt.plot(t, y_true[:n_plot, 0], marker="o", label="true (step 1)")
    plt.plot(t, preds[:n_plot, 0], marker="x", label="pred (step 1)")
    plt.xlabel("Sample index (in validation batch)")
    plt.ylabel("Target (first horizon step)")
    plt.title("Multi-step forecast (first horizon step)")

plt.grid(True)
plt.legend()
plt.show()

## 8. Save model artifacts

We now save the trained model to the `models` directory defined in the config, along with enough metadata to reproduce or reload the run later.

This mirrors what your time-series training script (`train_ts_mlp.py` or similar) would do in a non-notebook context.

In [ ]:
paths.models_dir.mkdir(parents=True, exist_ok=True)
model_path = paths.models_dir / f"ts_gru_{cfg.experiment_name}.pt"

torch.save({
    "model_state_dict": model.state_dict(),
    "model_hparams": {
        "input_dim": n_features,
        "hidden_dim": hidden_dim,
        "num_layers": num_layers,
        "horizon": horizon,
        "lookback": lookback,
    },
    "config": cfg_dict,
}, model_path)

LOGGER.info("Saved time-series model to: %s", model_path)
model_path

## 9. (Optional) Enable MLflow tracking

If you installed the `mlops` extra and configured MLflow, you can track this training run using `ml_tabular.mlops.mlflow_utils`.

This section is **optional**; comment it out if you don’t have MLflow set up yet.

In [ ]:
from ml_tabular.mlops.mlflow_utils import (
    is_mlflow_available,
    mlflow_run,
    log_params,
    log_metrics,
    log_artifact,
)
import os

if is_mlflow_available():
    import mlflow

    tracking_uri = os.getenv("MLFLOW_TRACKING_URI") or (paths.base_dir / "mlruns").as_uri()
    experiment_name = cfg.experiment_name + "_ts"

    with mlflow_run(
        enabled=True,
        experiment_name=experiment_name,
        run_name="time_series_quickstart",
        tracking_uri=tracking_uri,
        tags={"template": "ml_tabular", "notebook": "10_time_series_quickstart"},
    ):
        # Log run-level parameters
        log_params({
            "model_type": "GRUForecaster",
            "input_dim": n_features,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers,
            "horizon": horizon,
            "lookback": lookback,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
        })

        # Optionally re-run a short training (or reuse history from above)
        # Here we reuse the already-trained model and history
        log_metrics({
            "final_train_loss": float(history["train_losses"][-1]),
            "final_val_loss": float(history["val_losses"][-1]),
        })

        if model_path.exists():
            log_artifact(model_path, artifact_path="models")
else:
    print("MLflow not available; skipping MLflow tracking demo.")

## 10. Summary

In this time-series quickstart, you:

- Loaded configuration via the same `AppConfig` system used for tabular
- Parsed and sorted time-indexed data, with optional series IDs
- Converted the dataset into supervised sequences using `TimeSeriesSequenceDataset`
- Built a small GRU-based model for forecasting
- Trained with shared utilities (`fit`, `EarlyStopping`) and visualized loss curves
- Saved model artifacts and (optionally) logged them to MLflow

This reinforces the core point of your template: **a consistent, professional workflow** across modalities (tabular and time-series), with clean separation between:

- Config
- Data & datasets
- Models
- Training loops
- MLOps / experiment tracking

From here, you can swap in more advanced models (Temporal CNNs, Transformers, etc.) without changing the surrounding plumbing.